In [4]:
import pandas as pd
import numpy as np

In [28]:
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTQEI2bS5E6GBqa9U3cWf9Hddje62AbkGUonNJWLEhKHeBMWJTWZ9u_Mva1dVtPizi6XyKK-h2KGz2P/pub?gid=0&single=true&output=csv"
df_raw = pd.read_csv(
    url,
    delimiter=',',
    engine='python',
    on_bad_lines='skip'
)
df_raw.head()

,order_id,date,customer_name,product,quantity,price,total,city
0,1,2024/01/03,john doe,Laptop,1,10OO,1000.0,jakarta
1,2,03-01-2024,Jane Doe,mouse,two,100,200.0,bandung
2,3,Jan 04 2024,budi,KEYBOARD,1,500,NaN,jakarta
3,4,2024.01.05,Alice,Monitor,1,2OO,200.0,surabaya
4,5,2024-01-05,john doe,Laptop,-1,1000,-1000.0,jakarta


In [70]:
import re
import pandas as pd
from dateutil import parser
import numpy as np
df = df_raw.copy()


def robust_parse_date(x):
    if pd.isna(x):
        return pd.NaT
    s = str(x).strip()
    s = re.sub(r"[\/\.]", "-", s)
    s = s.replace(",", " ").strip()
    fmts = [
        "%Y-%m-%d",   # 2024-01-03
        "%d-%m-%Y",   # 03-01-2024
        "%m-%d-%Y",   # 01-03-2024 
        "%b %d %Y",   # Jan 04 2024
        "%B %d %Y",   # January 04 2024
        "%d %b %Y",   # 04 Jan 2024
        "%Y-%d-%m"    # rare swaps
    ]
    for f in fmts:
        try:
            return pd.to_datetime(s, format=f)
        except Exception:
            pass
    try:
        return parser.parse(s, dayfirst=True, fuzzy=True)
    except Exception:
        return pd.NaT

df["date_clean"] = df["date"].apply(robust_parse_date)

df[["date", "date_clean"]].assign(date_clean_only = lambda d: d["date_clean"].dt.date)




,date,date_clean,date_clean_only
0,2024/01/03,2024-01-03,2024-01-03
1,03-01-2024,2024-01-03,2024-01-03
2,Jan 04 2024,2024-01-04,2024-01-04
3,2024.01.05,2024-01-05,2024-01-05
4,2024-01-05,2024-01-05,2024-01-05


In [85]:
import re
import numpy as np
import pandas as pd

word_to_num = {
    "zero": 0,
    "one": 1,
    "two": 2,
    "three": 3,
    "four": 4,
    "five": 5,
    "six": 6,
    "seven": 7,
    "eight": 8,
    "nine": 9,
    "ten": 10
}

def clean_quantity(x):
    if pd.isna(x):
        return np.nan

    s = str(x).strip().lower()

    if s in word_to_num:
        return word_to_num[s]

    s = s.replace("o", "0")  
    s = s.replace(" ", "")    

    if s in ["", "-", "--"]:
        return np.nan

    s_clean = re.sub(r"[^0-9-]", "", s)

    if not re.fullmatch(r"-?\d+", s_clean):
        return np.nan

    val = int(s_clean)

    return abs(val)

df["quantity_clean"] = df["quantity"].apply(clean_quantity)

df[["quantity", "quantity_clean"]]


,quantity,quantity_clean
0,1,1
1,two,2
2,1,1
3,1,1
4,-1,1


In [88]:
def clean_price(x):
    if pd.isna(x): return None
    x = str(x).strip()

    x = x.replace("O", "0").replace("o", "0")
    x = x.replace(",", "")

    try:
        return float(x)
    except:
        return None

df["price_clean"] = df["price"].apply(clean_price)
df

,order_id,date,customer_name,product,quantity,price,total,city,date_clean,price_clean,quantity_clean
0,1,2024/01/03,john doe,Laptop,1,10OO,1000.0,jakarta,2024-01-03,1000.0,1
1,2,03-01-2024,Jane Doe,mouse,two,100,200.0,bandung,2024-01-03,100.0,2
2,3,Jan 04 2024,budi,KEYBOARD,1,500,NaN,jakarta,2024-01-04,500.0,1
3,4,2024.01.05,Alice,Monitor,1,2OO,200.0,surabaya,2024-01-05,200.0,1
4,5,2024-01-05,john doe,Laptop,-1,1000,-1000.0,jakarta,2024-01-05,1000.0,1


In [94]:
df["total_clean"] = df["quantity_clean"] * df["price_clean"]
df

,order_id,date,customer_name,product,quantity,price,total,city,date_clean,price_clean,quantity_clean,total_clean
0,1,2024/01/03,john doe,Laptop,1,10OO,1000.0,jakarta,2024-01-03,1000.0,1,1000.0
1,2,03-01-2024,Jane Doe,mouse,two,100,200.0,bandung,2024-01-03,100.0,2,200.0
2,3,Jan 04 2024,budi,KEYBOARD,1,500,NaN,jakarta,2024-01-04,500.0,1,500.0
3,4,2024.01.05,Alice,Monitor,1,2OO,200.0,surabaya,2024-01-05,200.0,1,200.0
4,5,2024-01-05,john doe,Laptop,-1,1000,-1000.0,jakarta,2024-01-05,1000.0,1,1000.0


In [103]:
df["product_clean"] = df["product"].str.strip().str.lower()
df["customer_clean"] = df["customer_name"].str.strip().str.title()
df["city_clean"] = df["city"].str.strip().str.title()
df

,order_id,date,customer_name,product,quantity,price,total,city,date_clean,price_clean,quantity_clean,total_clean,product_clean,customer_clean,city_clean
0,1,2024/01/03,john doe,Laptop,1,10OO,1000.0,jakarta,2024-01-03,1000.0,1,1000.0,laptop,John Doe,Jakarta
1,2,03-01-2024,Jane Doe,mouse,two,100,200.0,bandung,2024-01-03,100.0,2,200.0,mouse,Jane Doe,Bandung
2,3,Jan 04 2024,budi,KEYBOARD,1,500,NaN,jakarta,2024-01-04,500.0,1,500.0,keyboard,Budi,Jakarta
3,4,2024.01.05,Alice,Monitor,1,2OO,200.0,surabaya,2024-01-05,200.0,1,200.0,monitor,Alice,Surabaya
4,5,2024-01-05,john doe,Laptop,-1,1000,-1000.0,jakarta,2024-01-05,1000.0,1,1000.0,laptop,John Doe,Jakarta


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=10e9380b-59c9-487d-b4e4-21012e1e2b10' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>